In [1]:
%matplotlib qt
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import save_pkl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)

Failed loading module nndac:  inconsistent use of tabs and spaces in indentation (nndac.py, line 32)


In [2]:
Qubits = {'1':{'r':{'Fr': 7.325e9, 'P': -55},
               'q':{'00-1-01': 4.246158e9, 
                    }},
        '2':{'r':{'Fr': 7.625e9, 'P':-50}, 
               'q':{'00-1-01': 4.250412e9,#4.24947e9,
                    }},
         '3':{'r':{'Fr': 8.35e9, 'P':-50}, 
               'q':{'00-1-01': 4.250412e9,#4.24947e9,
                    }}}
qubit_params = Qubits
qubit_id = '1'

In [2]:
# RF switch for making sure we know what sample we are measuring
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
# lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')
pna.set_sweep_mode("LIN")

CHASSIS = 0
SLOT_AWG1 = 3
SLOT_AWG2 = 5

awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
awg1 = Keysight_M3202A_S('awg1', CHASSIS, SLOT_AWG1)
awg2 = Keysight_M3202A_S('awg2', CHASSIS, SLOT_AWG2)

adc = instruments.Spectrum_M4i22xx('adc')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

## Single-tone spectroscopy 

In [4]:
#Stuff for IQ-mixers opening
awg1.trigger_source_types = [6,6,6,6]
awg1.trigger_source_channels = [4000, 4000,4000,4000]
awg1.trigger_delays = [7+10,7+10,7+10,7+10] # wait 60 ns for sync with awg2
awg1.trigger_behaviours = [4,4,4,4]
awg1.stop()
for channel in range(0,1):
    awg1.set_amplitude(0.03, channel=channel)
    awg1.set_offset(0.6,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform =[0]*50000, channel=channel)
awg1.set_marker(length=200, delay=0, channel=0, pxi_channels=2)
awg1.run()

In [ ]:
lo1.set_status(0)
pna.set_status(1)
#pna.set_power(-35) #
powers = np.linspace(10, -30, 41)
pna.set_nop(401)
pna.set_xlim(Qubits[qubit_id]['r']['Fr']-10e6, Qubits[qubit_id]['r']['Fr']+100e6)
pna.set_bandwidth(100)
#freqs = pna.get_freqpoints()
#S21 = pna.get_tracedata()
meas = sweep.sweep(pna, (powers, pna.set_power, 'Мощность, дБ'), filename='Однотоновая спектроскопия, образец #{}'.format(qubit_id))

In [13]:
lo1.set_status(0)
nndac_channel_id = 0
pna.set_power(-40)
pna.set_nop(601)
pna.set_bandwidth(100)
voltage_max=2
voltage_nop = 61

voltages=np.linspace(1.65,1.75,voltage_nop)
pna.set_xlim(7.3e9,7.50e9)
try:
    result = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)), 
                        filename=' anticrossings sample 2 qubit 1{} Icoil power {}'.format(qubit_id, pna.get_power()))
finally:
    dac.set_voltage(0, channel=nndac_channel_id)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 10 2018 05:21:21


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 10644

Elapsed time: 0 h 5 m 39.69 s


## Two-tone spectroscopy

In [8]:
# Двухтоновка в пи
ex_freqs = np.arange(0.01e9, 0.8e9, 2e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(-10) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.355e9)
pna.set_frequency(7.355e9)
pna.set_bandwidth(10)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-20)
voltage_max=2
voltage_nop = 101
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(1.45,1.95,voltage_nop)
header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, 
                       (voltages, lambda x: dac.set_voltage(x, channel=0), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Двухтоновая спектроскопия образец #{}, power lo {},power pna {}".format(qubit_id,\
                                                                                                          lo1.get_power(),
                                                                                                         pna.get_power()))
except KeyboardInterrupt: pass 
lo1.set_status(0)


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 08 2018 11:19:00


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 5736

Elapsed time: 1 h 20 m 47.32 s


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 280743: character maps to <undefined>

In [59]:
# Двухтоновка в пи
ex_freqs = np.arange(0.1e9, 0.4e9, 2e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(-5) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.355e9)
pna.set_frequency(7.355e9)
pna.set_bandwidth(5)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-20)
voltage_max=2
voltage_nop = 101
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(1.55,1.85,voltage_nop)
powers = range(-14,-5,1)
for power in powers:
    lo1.set_power(power)
    ex1D = sweep.sweep(pna, 
                       (voltages, lambda x: dac.set_voltage(x, channel=0), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Second tone{}, power lo {},power pna {}".format(qubit_id,\
                                                                                      power,
                                                                                     pna.get_power()))

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 08 2018 18:15:32


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


...

Exiting sweep main thread #{} via return 2636

Elapsed time: 0 h 53 m 4.08 s


HTML(value='')

Started at:  Dec 09 2018 01:20:42


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 9220

Elapsed time: 0 h 53 m 2.97 s


In [58]:
ex_freqs = np.arange(0.1e9, 0.35e9, 2e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(-5) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.355e9)
pna.set_frequency(7.355e9)
pna.set_bandwidth(5)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-20)
voltage_max=2
voltage_nop = 101
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(1.55,1.85,voltage_nop)
powers = range(-20,-16,1)
for power in powers:
    lo1.set_power(power)
    ex1D = sweep.sweep(pna, 
                       (voltages, lambda x: dac.set_voltage(x, channel=0), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Second tone{}, power lo {},power pna {}".format(qubit_id,\
                                                                                      power,
                                                                                     pna.get_power()))

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 08 2018 15:18:19


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


...

Exiting sweep main thread #{} via return 6868

Elapsed time: 0 h 44 m 13.89 s


HTML(value='')

Started at:  Dec 08 2018 17:31:12


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 11696

Elapsed time: 0 h 44 m 15.71 s


In [9]:
# Двухтоновка в пи
ex_freqs = np.arange(0.1e9, 0.4e9, 1e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(-14) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
#pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
#pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_centerfreq(7.355e9)
pna.set_frequency(7.355e9)
pna.set_bandwidth(5)
#pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_power(-20)
voltage_max=2
voltage_nop = 201
# voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
voltages=np.linspace(1.55,1.85,voltage_nop)
powers = [-20]
for power in powers:
    pna.set_power(power)
    ex1D = sweep.sweep(pna, 
                       (voltages, lambda x: dac.set_voltage(x, channel=0), 'Coil #{} voltage,qubit{}'.format(1,qubit_id)),
                       (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'),
                       filename = "Second tone{}, power lo {},power pna {}".format(qubit_id,\
                                                                                     lo1.get_power(),
                                                                                      power))

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Dec 09 2018 07:31:07


C:\qtlab_replacement\qsweepy\sweep.py:218: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data[mname][[(i) for i in indeces]+[...]] = mpoint_m


Exiting sweep main thread #{} via return 3732

Elapsed time: 3 h 31 m 2.15 s


In [15]:
import pickle as pkl

In [18]:
filename = r"C:\Data\2018-12-08\12-58-30-Двухтоновая спектроскопия образец #1, power lo -12.0,power pna -20.0\Двухтоновая спектроскопия образец #1, power lo -12.0,power pna -20.0.pkl"
f = open(filename, 'rb')
data = pkl.load(f)

In [31]:
S21 = data[1]['S-parameter'][2]
voltages = data[1]['S-parameter'][1][0]
freqs = data[1]['S-parameter'][1][1]

In [41]:
n=55
voltages[n]

1.715

In [42]:
# plt.figure()
plt.plot(freqs,np.imag(S21[n]))
plt.show()